In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Connection parameters
host = 'evchargerdb.cjlsbfswobs8.us-east-1.rds.amazonaws.com'
database = 'evchargerinst'
user = 'postgres'
password = 'charger2024'
port = '5432'

# Establishing a connection
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection to PostgreSQL database successful")
except psycopg2.Error as e:
    print("Error: Could not connect to PostgreSQL database")
    print(e)

Connection to PostgreSQL database successful


In [4]:
# Define the ALTER TABLE query
geo_query = 'ALTER TABLE "evChargers".tristatecharingstations ADD COLUMN geom2 GEOGRAPHY(Point, 4326);'

# Execute the ALTER TABLE query
try:
    cursor = conn.cursor()
    cursor.execute(geo_query)
    conn.commit()  # Committing the transaction to make the changes persistent
    print("Column 'geom' added successfully.")
    
except psycopg2.Error as e:
    conn.rollback()  # Rolling back the transaction in case of error
    print("Error executing the query:", e)

# Define the SELECT query to fetch data from the table
select_query = 'SELECT * FROM "evChargers".tristatecharingstations;'

Column 'geom' added successfully.


In [5]:
df = pd.read_sql_query(select_query, conn)

C:\Users\alawr\AppData\Local\Temp\ipykernel_17804\1010624936.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(select_query, conn)


In [6]:
df.head()

,station name,street address,city,state,zipcode,latitude,longitude,ev network,facility type,geom,geom2
0,Element Hotels - Ewing,1000 Sam Weinroth Rd,Ewing,NJ,08628,40.286520,-74.80752,Non-Networked,HOTEL,0101000020E61000006BB75D68AEB352C031D3F6AFAC24...,None
1,Giant Eagle #43 - Pine Township,155 Towne Center Dr,Wexford,PA,15090,40.632137,-80.05428,Non-Networked,GROCERY,0101000020E6100000EC34D252790354C0B8CB7EDDE950...,None
2,Giant Eagle - Market District Settlers Ridge,100 Settlers Ridge Centre Dr,Pittsburgh,PA,15205,40.441450,-80.14975,Non-Networked,GROCERY,0101000020E6100000DD240681950954C0DC68006F8138...,None
3,Elizabeth Parking Authority - J Christian Boll...,37 Caldwell Pl,Elizabeth,NJ,07201,40.663850,-74.21658,Non-Networked,PARKING_GARAGE,0101000020E6100000EE3D5C72DC8D52C08CB96B09F954...,None
4,Eaton Corp,1000 Cherrington Pkwy,Moon,PA,15108,40.492880,-80.20018,Non-Networked,OFFICE_BLDG,0101000020E61000000D54C6BFCF0C54C01E6D1CB1163F...,None


In [7]:
list(df.columns)

['station name',
 'street address',
 'city',
 'state',
 'zipcode',
 'latitude',
 'longitude',
 'ev network',
 'facility type',
 'geom',
 'geom2']

In [9]:
# nj_rows = df[df['state'] == 'NJ']
# nj_rows['zipcode'] = nj_rows['zipcode'].apply(lambda x: x.zfill(5))
# df.update(nj_rows)

In [10]:
# cleaned_df = df[df['facility type'] != '']
# cleaned_df.head()

In [11]:
# selected_columns = cleaned_df[['station name',
#  'street address',
#  'city',
#  'state',
#  'zipcode',
#  'country',
#  'facility type',
#  'station phone',
#  'latitude',
#  'longitude',
#  'geom',
#  'access days time',
#  'charger count',
#  'ev network',
# 'ev other info',
#  'ev connector types',
#  'ev pricing']]
# updated_df = selected_columns
# updated_df.head()

In [11]:
# # SQL to create the view
# table_name = 'evcharingstations_all_2'
# schema_name = 'evchargers'

In [12]:
selected_columns = df[['station name',
 'street address',
 'city',
 'state',
 'zipcode',
 'latitude',
 'longitude',
 'geom',
 'ev network',
 'facility type']]
updated_df = selected_columns
updated_df.head()

,station name,street address,city,state,zipcode,latitude,longitude,geom,ev network,facility type
0,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040540,-118.271385,0101000020E6100000000000605E915DC0000000603005...,Non-Networked,PARKING_GARAGE
1,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,34.068720,-118.064000,0101000020E6100000000000A018845DC0000000E0CB08...,Non-Networked,STATE_GOV
2,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,32.899470,-117.243000,0101000020E6100000000000408D4F5DC0000000E02173...,Non-Networked,HOSPITAL
3,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,34.221664,-118.468370,0101000020E6100000000000C0F99D5DC0000000805F1C...,Non-Networked,CAR_DEALER
4,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,33.909912,-117.459050,0101000020E610000000000020615D5DC00000000078F4...,Non-Networked,SHOPPING_MALL


In [13]:
tristate_df = updated_df[updated_df['state'].isin(['NJ', 'NY', 'PA'])]
tristate_df.head()

,station name,street address,city,state,zipcode,latitude,longitude,geom,ev network,facility type
86,Element Hotels - Ewing,1000 Sam Weinroth Rd,Ewing,NJ,08628,40.286520,-74.80752,0101000020E610000000000060AEB352C0000000A0AC24...,Non-Networked,HOTEL
383,Giant Eagle #43 - Pine Township,155 Towne Center Dr,Wexford,PA,15090,40.632137,-80.05428,0101000020E610000000000060790354C0000000E0E950...,Non-Networked,GROCERY
428,Giant Eagle - Market District Settlers Ridge,100 Settlers Ridge Centre Dr,Pittsburgh,PA,15205,40.441450,-80.14975,0101000020E610000000000080950954C0000000608138...,Non-Networked,GROCERY
447,Elizabeth Parking Authority - J Christian Boll...,37 Caldwell Pl,Elizabeth,NJ,07201,40.663850,-74.21658,0101000020E610000000000080DC8D52C000000000F954...,Non-Networked,PARKING_GARAGE
449,Eaton Corp,1000 Cherrington Pkwy,Moon,PA,15108,40.492880,-80.20018,0101000020E6100000000000C0CF0C54C0000000C0163F...,Non-Networked,OFFICE_BLDG


In [14]:
tristate_df.count()

station name      1713
street address    1713
city              1713
state             1713
zipcode           1713
latitude          1713
longitude         1713
geom              1713
ev network        1713
facility type     1713
dtype: int64

In [8]:

# Construct the connection string
db_connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Define the schema name and view name
schema_name = 'evChargers'
table_name = 'tristatecharingstations_v2'

# Write the DataFrame as a view to the database
df.to_sql(table_name, con=create_engine(db_connection_str), schema=schema_name, if_exists='replace', index=False)


713